IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M2\FAULT_M2_5


VOLO m2 FAULT 5%

In [3]:
rcou_m2_fault5 = pd.read_csv("RCOU.csv")
rcou_m2_fault5 = rcou_m2_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

rcou_m2_fault5 = rcou_m2_fault5[((rcou_m2_fault5['C9'] >= 1400) & (rcou_m2_fault5['C10'] >= 1400) & (rcou_m2_fault5['C11'] >= 1400) & (rcou_m2_fault5['C12'] >= 1400) & (rcou_m2_fault5['C13'] >= 1400) & (rcou_m2_fault5['C14']>= 1400))]

pd.set_option('display.max_rows',700)
display(rcou_m2_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
2420,70614501,1416,1480,1456,1442,1422,1475
2421,70628999,1448,1463,1501,1407,1494,1416
2422,70639526,1458,1465,1492,1429,1488,1434
2423,70649667,1467,1467,1477,1456,1476,1457
2424,70660471,1482,1462,1489,1455,1497,1446
...,...,...,...,...,...,...,...
7938,131338225,1525,1447,1519,1454,1463,1510
7939,131348455,1514,1459,1534,1436,1464,1509
7940,131358716,1506,1466,1525,1446,1443,1527
7941,131368599,1519,1450,1526,1441,1457,1513


In [5]:
xkf1_m2_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m2_fault5 = xkf1_m2_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m2_fault5 = xkf1_m2_fault5[((xkf1_m2_fault5['TimeUS'] >= 70614501) & (xkf1_m2_fault5['TimeUS'] <= 131379386))]
print(xkf1_m2_fault5)

         TimeUS  Roll  Pitch    Yaw
2423   70626950  0.80  -0.96  22.96
2424   70637199  0.79  -0.98  22.96
2425   70647273  0.78  -0.98  22.97
2426   70657896  0.78  -1.00  22.97
2427   70668621  0.75  -1.02  22.98
...         ...   ...    ...    ...
7944  131335984 -1.93   3.05  22.04
7945  131346546 -1.94   2.95  22.14
7946  131356703 -1.94   2.88  22.24
7947  131366708 -1.93   2.76  22.34
7948  131376923 -1.88   2.59  22.41

[5526 rows x 4 columns]


In [6]:
att_m2_fault5 = pd.read_csv("ATT.csv")
att_m2_fault5 = att_m2_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m2_fault5 = att_m2_fault5[((att_m2_fault5['TimeUS'] >= 70614501) & (att_m2_fault5['TimeUS'] <= 131379386))]
print(att_m2_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
2423   70628393     0.03  0.80      0.10  -0.96   22.93  22.96
2424   70638624     0.03  0.79      0.10  -0.98   22.93  22.96
2425   70648917     0.03  0.78      0.11  -0.98   22.93  22.97
2426   70659553     0.03  0.78      0.12  -1.00   22.93  22.97
2427   70670344     0.03  0.75      0.13  -1.02   22.93  22.98
...         ...      ...   ...       ...    ...     ...    ...
7947  131337507     1.82 -1.93      0.58   3.05   25.39  22.04
7948  131347650     1.88 -1.94      0.52   2.95   25.39  22.14
7949  131358142     1.94 -1.94      0.46   2.88   25.39  22.24
7950  131367962     2.01 -1.93      0.41   2.76   25.39  22.34
7951  131378544     2.06 -1.88      0.37   2.59   25.38  22.41

[5529 rows x 7 columns]


In [7]:
att_m2_fault5 = pd.merge_asof(att_m2_fault5,xkf1_m2_fault5, on="TimeUS", by=["Roll","Pitch","Yaw"], tolerance=100000)
print(att_m2_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      70628393     0.03  0.80      0.10  -0.96   22.93  22.96
1      70638624     0.03  0.79      0.10  -0.98   22.93  22.96
2      70648917     0.03  0.78      0.11  -0.98   22.93  22.97
3      70659553     0.03  0.78      0.12  -1.00   22.93  22.97
4      70670344     0.03  0.75      0.13  -1.02   22.93  22.98
...         ...      ...   ...       ...    ...     ...    ...
5524  131337507     1.82 -1.93      0.58   3.05   25.39  22.04
5525  131347650     1.88 -1.94      0.52   2.95   25.39  22.14
5526  131358142     1.94 -1.94      0.46   2.88   25.39  22.24
5527  131367962     2.01 -1.93      0.41   2.76   25.39  22.34
5528  131378544     2.06 -1.88      0.37   2.59   25.38  22.41

[5529 rows x 7 columns]


In [8]:
esc_0_m2_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m2_fault5 = esc_0_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m2_fault5 = esc_0_m2_fault5[((esc_0_m2_fault5['TimeUS'] >= 70614501) & (esc_0_m2_fault5['TimeUS'] <= 131379386))]
print(esc_0_m2_fault5)

        TimeUS   RPM  Curr
223   70703915  4701  2.93
224   70817407  5211  3.81
225   70922454  5591  3.96
226   71145483  6069  3.65
227   71244897  6049  4.06
228   71359670  6233  4.09
229   71470483  6285  4.19
230   71575135  6285  4.05
231   71686439  6291  4.40
232   71903879  6403  4.72
233   72008122  6553  4.85
234   72228618  6470  4.54
235   72343571  6425  3.81
236   72451541  6096  3.48
237   72560229  6050  3.90
238   72666059  5979  3.64
239   72771793  5860  4.02
240   72879936  6084  3.97
241   73102423  6047  4.16
242   73314348  6156  3.87
243   73434157  6136  3.98
244   73530423  6104  4.07
245   73639646  6212  4.19
246   73755036  6191  4.13
247   73857693  6256  4.28
248   73965804  6240  4.28
249   74076807  6172  4.42
250   74186709  6260  4.38
251   74289586  6344  4.63
252   74405633  6459  4.36
253   74511666  6307  5.00
254   74623312  6482  5.05
255   74733141  6486  5.36
256   74843198  6760  5.03
257   74951546  6660  5.08
258   75056606  6702  4.97
2

In [9]:
imu_2_m2_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m2_fault5 = imu_2_m2_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m2_fault5 = imu_2_m2_fault5[((imu_2_m2_fault5['TimeUS'] >= 70614501) & (imu_2_m2_fault5['TimeUS'] <= 131379386))]
print(imu_2_m2_fault5)

          TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ
9573    70616375 -0.055046 -0.052377 -0.011207 -0.166060  0.273514  -9.717393
9574    70621789  0.089052  0.031634 -0.010612 -0.258759  0.174801 -10.251830
9575    70628180 -0.002446 -0.056335 -0.023127 -0.291703 -0.011755 -10.056950
9576    70631689  0.029754 -0.033931 -0.008155 -0.316784 -0.110613 -10.065700
9577    70633488 -0.017690  0.042331  0.008342 -0.330894 -0.195240 -10.101020
...          ...       ...       ...       ...       ...       ...        ...
30548  131367802  0.011825 -0.207308  0.156284  0.600088  0.554415 -10.538300
30549  131370473  0.002054 -0.202995  0.160413  0.607579  0.646880 -10.524500
30550  131373093  0.022602 -0.259851  0.147129  0.591488  0.759371 -10.474610
30551  131376133  0.045938 -0.286306  0.137208  0.552823  0.840802 -10.579710
30552  131378441  0.082656 -0.258289  0.124889  0.530554  0.954088 -10.965860

[20980 rows x 7 columns]


In [10]:
m2_fault5 = pd.merge_asof(att_m2_fault5, imu_2_m2_fault5, on="TimeUS",  tolerance= 100000)
print(m2_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw      GyrX  \
0      70628393     0.03  0.80      0.10  -0.96   22.93  22.96 -0.002446   
1      70638624     0.03  0.79      0.10  -0.98   22.93  22.96 -0.012227   
2      70648917     0.03  0.78      0.11  -0.98   22.93  22.97 -0.013179   
3      70659553     0.03  0.78      0.12  -1.00   22.93  22.97  0.006291   
4      70670344     0.03  0.75      0.13  -1.02   22.93  22.98 -0.057392   
...         ...      ...   ...       ...    ...     ...    ...       ...   
5524  131337507     1.82 -1.93      0.58   3.05   25.39  22.04  0.026053   
5525  131347650     1.88 -1.94      0.52   2.95   25.39  22.14 -0.004182   
5526  131358142     1.94 -1.94      0.46   2.88   25.39  22.24 -0.031159   
5527  131367962     2.01 -1.93      0.41   2.76   25.39  22.34  0.011825   
5528  131378544     2.06 -1.88      0.37   2.59   25.38  22.41  0.082656   

          GyrY      GyrZ      AccX      AccY       AccZ  
0    -0.056335 -0.023127 -0.2

In [11]:
m2_fault5 = pd.merge_asof(esc_0_m2_fault5, m2_fault5, on="TimeUS",  tolerance= 100000)
print(m2_fault5)

        TimeUS   RPM  Curr  DesRoll   Roll  DesPitch  Pitch  DesYaw    Yaw  \
0     70703915  4701  2.93     0.04   0.68      0.18  -1.11   22.93  23.00   
1     70817407  5211  3.81     0.04   0.62      0.40  -1.28   22.93  23.09   
2     70922454  5591  3.96     0.00   0.39      0.84  -0.94   22.93  23.58   
3     71145483  6069  3.65    -0.66   1.00      2.14   1.69   22.93  26.31   
4     71244897  6049  4.06    -1.10   1.38      2.38   2.31   22.92  27.64   
5     71359670  6233  4.09    -1.84   1.18      2.61   2.06   22.92  28.94   
6     71470483  6285  4.19    -2.55   0.51      2.94   1.61   22.91  29.85   
7     71575135  6285  4.05    -3.27  -0.21      3.13   1.81   22.69  30.39   
8     71686439  6291  4.40    -3.76  -1.28      3.25   2.70   21.19  30.66   
9     71903879  6403  4.72    -3.92  -1.79      3.38   3.23   16.02  29.19   
10    72008122  6553  4.85    -3.82  -1.57      2.93   2.99   13.70  27.59   
11    72228618  6470  4.54    -3.34  -1.51      2.64   2.60   10

In [12]:
m2_fault5["TimeUS"] = m2_fault5["TimeUS"] - m2_fault5.iloc[0]["TimeUS"]
m2_fault5["TimeUS"] = m2_fault5["TimeUS"].astype(int)
m2_fault5["Guasto"] = 1
print(m2_fault5)

       TimeUS   RPM  Curr  DesRoll   Roll  DesPitch  Pitch  DesYaw    Yaw  \
0           0  4701  2.93     0.04   0.68      0.18  -1.11   22.93  23.00   
1      113492  5211  3.81     0.04   0.62      0.40  -1.28   22.93  23.09   
2      218539  5591  3.96     0.00   0.39      0.84  -0.94   22.93  23.58   
3      441568  6069  3.65    -0.66   1.00      2.14   1.69   22.93  26.31   
4      540982  6049  4.06    -1.10   1.38      2.38   2.31   22.92  27.64   
5      655755  6233  4.09    -1.84   1.18      2.61   2.06   22.92  28.94   
6      766568  6285  4.19    -2.55   0.51      2.94   1.61   22.91  29.85   
7      871220  6285  4.05    -3.27  -0.21      3.13   1.81   22.69  30.39   
8      982524  6291  4.40    -3.76  -1.28      3.25   2.70   21.19  30.66   
9     1199964  6403  4.72    -3.92  -1.79      3.38   3.23   16.02  29.19   
10    1304207  6553  4.85    -3.82  -1.57      2.93   2.99   13.70  27.59   
11    1524703  6470  4.54    -3.34  -1.51      2.64   2.60   10.74  22.71   

In [13]:
path_file = path_file.replace(r"m2\FAULT_m2_5", "")
os.chdir(path_file)
m2_fault5.to_csv('m2_fault5.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M2\FAULT_M2_5
